In [ ]:
# Basic imports
import os
import sys
import cv2 
import numpy as np  
import pandas as pd 
import json
import openslide

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from util import plot_artery_counts_by_case, barplot, distribution_analysis

sys.path.append(os.path.abspath('..'))
from utils.utils_data import get_veesel_sheets, get_segmentations
from utils.utils_geometry import get_contours_by_classification, is_contour_match_bounds, offset_contours

from utils.utils_survival import survival_analysis
from utils.utils_vis import gallery_view
from utils.utils_constants import (ARTERY_TYPES,
                                   DISEASE_TYPES,
                                   VESSEL_NEPTUNE_PAT_INFO_W_SCORE_PATH as VESSEL_PAT_INFO_W_SCORE_PATH,
                                   CLASSIFICATION_PATH,
                                   TISSUE_SEGMENTATION_REGION_PATH,
                                   SEGMENTATION_DIR,
                                   COMBINED_CLASSIFICATION_PATH,
                                   ANALYSIS_DOC_PATH,
                                   TRI_CASE_DIR,
                                   CROPPED_VESSELS_DIR)

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
pat_df = pd.read_csv(VESSEL_PAT_INFO_W_SCORE_PATH)
# pat_df['DaysBXtoESRDorEGFR40_LR'] = pd.to_numeric(pat_df['DaysBXtoESRDorEGFR40_LR'], errors='coerce')
pat_df['ESRDorEGFR40BX_LR'] = pat_df['ESRDorEGFR40BX_LR'].map({'1: Yes': 1, '0: No': 0}).astype(int)
pat_df = pat_df[pat_df["Num_All_Arteries"] > 0]

In [ ]:
def is_contour_intersecting_or_within(cnt_iner, cnt_outer):
    """Determine if an inner contour intersects or is completely within an outer contour."""
    return any(cv2.pointPolygonTest(cnt_outer, (int(point[0]), int(point[1])), False) >= 0 for point in cnt_iner)


def check_bbox_with_contour(bbox_x, bbox_y, bbox_width, bbox_height, cortex_contour):
    # Convert bounding box to contour
    rect_contour = np.array([
        [bbox_x, bbox_y],
        [bbox_x + bbox_width, bbox_y],
        [bbox_x + bbox_width, bbox_y + bbox_height],
        [bbox_x, bbox_y + bbox_height]
    ], dtype=np.int32).reshape((-1, 2))

    return is_contour_intersecting_or_within(rect_contour, cortex_contour)

def vis_slide(classifications, segmentations, cortex_contour=None):
    for artery_type in ARTERY_TYPES:
        print(artery_type)
        images = []
        titles = []
        for index, row in classifications[classifications["Artery Type"] == artery_type].iterrows():
            bbox_x, bbox_y, bbox_width, bbox_height = map(int, row["Bounding Box"].split(","))

            if cortex_contour is not None:
                is_inside = check_bbox_with_contour(bbox_x, bbox_y, bbox_width, bbox_height, cortex_contour)
                if not is_inside:
                    continue
            outer_contours = get_contours_by_classification(
                segmentations,
                lambda contour: is_contour_match_bounds(
                    contour, (bbox_x, bbox_y, bbox_width, bbox_height)
                ),
                "Media"
            )
            assert(len(outer_contours)) == 1
            # vessel_size = int(np.log(cv2.contourArea(outer_contours[0])))
            area = cv2.contourArea(outer_contours[0])
            vessel_size = f"{round(area):.1e}"

            # cv2.drawContours(slide, outer_contours, -1, [255, 0, 0], 2)  # Outer contour in red
            # Construct the path to the image file
            image_path = os.path.join(CROPPED_VESSELS_DIR, artery_type, 
                                    row["Image Name"].replace(".png", "_w_ann.png"))
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Correct function to load the image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct color display
            images.append(img)
            # Create a title using multiple fields from the DataFrame
            title = f"Size: {vessel_size}; AS: {row['Arteriosclerosis Severity']}; HS: {row['Hyalinosis Severity']}"
            # title = f"B{row['Image Name'].split('_')[1]}{row['Artery ID']}-AS: {row['Arteriosclerosis Severity']}; HS: {row['Hyalinosis Severity']}"
            titles.append(title)
        if len(images) > 0:
            gallery_view(images, titles, cols=4)
        

In [ ]:
def get_border_of_cnt(cnt, border=0):
    # cnt: coordinates, list of list, [[0, 0], [0, 1], ...]
    cnt = np.array(cnt, dtype=int).squeeze()
    xmin, xmax = np.min(cnt[:, 0]), np.max(cnt[:, 0])
    ymin, ymax = np.min(cnt[:, 1]), np.max(cnt[:, 1])
    return (xmin-border, xmax+border, ymin-border, ymax+border)

def get_border_of_ann(ann, border=50):
    # ann: need to be cleaned to get coordinates
    (xmin, xmax) = (float('inf'), 0)
    (ymin, ymax) = (float('inf'), 0)
    for (i, ann_i) in enumerate(ann):
#         coords_raw = ann_i['geometry']['coordinates']
        coords = ann_i['geometry']['coordinates']
        curr_xmin, curr_xmax, curr_ymin, curr_ymax = get_border_of_cnt(coords)
        xmin = min(xmin, curr_xmin)
        xmax = max(xmax, curr_xmax)
        ymin = min(ymin, curr_ymin)
        ymax = max(ymax, curr_ymax)

    return (xmin-border, xmax+border, ymin-border, ymax+border)

In [ ]:
# pat_df_selected = pat_df[pat_df['ESRDorEGFR40BX_LR'] == 1].nsmallest(40, 'DaysBXtoESRDorEGFR40_LR')

with open(TISSUE_SEGMENTATION_REGION_PATH, 'r') as file:
     tissue_segmentations = json.load(file)

available_sheetnames = pd.ExcelFile(CLASSIFICATION_PATH).sheet_names

count = 0
for i, (index, row) in enumerate(pat_df.iterrows()):
     if i!=1: continue
     slide_filename = row["WSI_Selected"]
     logging.info(f"Processing: {i+1}/{len(pat_df)}: {slide_filename}")

     slide_path = os.path.join(TRI_CASE_DIR, slide_filename)
     slide_0 = openslide.OpenSlide(slide_path)

     slide_basename = os.path.splitext(slide_filename)[0]
     classifications = get_veesel_sheets(CLASSIFICATION_PATH, slide_basename, available_sheetnames, remove_others=True)
     if len(classifications) > 25: continue
     if len(classifications) < 10: continue
     if len(np.unique(classifications["Artery Type"].values)) < 3: continue
     if len(classifications[classifications["Artery Type"] == "Arcuate Arteries"]) < 2: continue
     count += 1
     if count >= 5: break
     segmentations_path = os.path.join(SEGMENTATION_DIR, f"{slide_basename}.geojson")
     segmentations = get_segmentations(segmentations_path, clean=True)

     # xmin, xmax, ymin, ymax = get_border_of_ann(segmentations, border=0)
     # slide = slide_0.read_region((xmin, ymin), 0, (xmax-xmin, ymax-ymin))
     
     # level = min(4, len(slide_0.level_dimensions)-1)
     # downsample = slide_0.level_downsamples[level] 

     # # Adjust coordinates for the target level
     # xmin_adj = int(xmin / downsample)
     # xmax_adj = int(xmax / downsample)
     # ymin_adj = int(ymin / downsample)
     # ymax_adj = int(ymax / downsample)
     # width = xmax_adj - xmin_adj
     # height = ymax_adj - ymin_adj

     # Read the region from the specified level
     # slide = slide_0.read_region((xmin_adj, ymin_adj), level, (width, height))
     # slide = np.asarray(slide)
     # slide = cv2.cvtColor(slide, cv2.COLOR_RGBA2RGB)
     # cortex_contours = get_contours_by_classification(segmentations, filter_fn=lambda x: True, classification="Cortex")
     # cortex_contours = offset_contours(cortex_contours, (xmin, ymin))
     # # cortex_contours = [np.round(cnt / downsample).astype(np.int32).reshape(-1, 1, 2) for cnt in cortex_contours]
     # if len(cortex_contours) > 0:
     #      cv2.drawContours(slide, cortex_contours, -1, [255, 0, 0], 2)  # Outer contour in red
     
     # fig, ax = plt.subplots(figsize=(10, 10))
     # cax = ax.imshow(slide)
     # plt.axis("off")
     # plt.show()
     
     vis_slide(classifications, segmentations, cortex_contour=None)


     # if len(cortex_contours) == 0:
     #      continue
     
     # for cortex_contour in cortex_contours:
     #      x, y, w, h = cv2.boundingRect(cortex_contour)  
          
     #      slide = slide_0.read_region((x, y), 0, (w, h))
     #      slide = np.asarray(slide)
     #      slide = cv2.cvtColor(slide, cv2.COLOR_RGBA2RGB)

     #      slide = vis_slide(slide, slide_filename, classifications, segmentations, cortex_contour=cortex_contour)
     #      fig, ax = plt.subplots()
     #      cax = ax.imshow(slide)
     #      plt.axis("off")
     #      plt.show()

    # slide_basename = os.path.splitext(slide_filename)[0]
    # classifications = get_veesel_sheets(CLASSIFICATION_PATH, slide_basename, available_sheetnames, remove_others=True)
    # if classifications.empty:
    #     continue  # Skip to if no relevant data
    # vis_slide(slide_filename, classifications)